In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets

In [2]:
# This is needed to authenticate us to
import os
%load_ext dotenv
%dotenv
try:
    from requests_oauthlib import OAuth2
except ModuleNotFoundError:
    import sys
    import os

    # I need this because requests_oauth gets installed in a weird place on my system
    sys.path.append('/usr/local/lib/python3.6/site-packages')
    from requests_oauthlib import OAuth2

In [3]:
OAuth2

requests_oauthlib.oauth2_auth.OAuth2

In [4]:
os.getenv("CLIENT_ACCESS_TOKEN"), os.getenv("CLIENT_SECRET"), os.getenv("CLIENT_ID")

('mBnDnHyEI_KGNyYv39-7efdwjJXc5cWq77fg_eDlsO8RDkYS8uu0YvOiMPS0e-FX',
 '2oVQqSrDutfqBf4b1Vv_v7EsXOURvV4zInrTcYkH1o3Ih2zJL7ibUy3DnjG08L_egKNqJnlgxgjZwwR9AOLwDw',
 'tg16sSXJ--cefig_zx6WGTiAizP7C2gCxP3ss_51lY5IiK_tOs_yhYlGT6jh5XvX')

In [5]:
# %load ~/anaconda3/lib/python3.6/site-packages/requests_oauthlib/oauth2_auth.py
from __future__ import unicode_literals
from oauthlib.oauth2 import WebApplicationClient, InsecureTransportError
from oauthlib.oauth2 import is_secure_transport
from requests.auth import AuthBase


class OAuth2(AuthBase):
    """Adds proof of authorization (OAuth2 token) to the request."""

    def __init__(self, client_id=None, client=None, token=None):
        """Construct a new OAuth 2 authorization object.

        :param client_id: Client id obtained during registration
        :param client: :class:`oauthlib.oauth2.Client` to be used. Default is
                       WebApplicationClient which is useful for any
                       hosted application but not mobile or desktop.
        :param token: Token dictionary, must include access_token
                      and token_type.
        """
        self._client = client or WebApplicationClient(client_id, token=token)
        if token:
            for k, v in token.items():
                setattr(self._client, k, v)

    def __call__(self, r):
        """Append an OAuth 2 token to the request.

        Note that currently HTTPS is required for all requests. There may be
        a token type that allows for plain HTTP in the future and then this
        should be updated to allow plain HTTP on a white list basis.
        """
        if not is_secure_transport(r.url):
            raise InsecureTransportError()
        r.url, r.headers, r.body = self._client.add_token(r.url,
                http_method=r.method, body=r.body, headers=r.headers)
        return r


In [6]:
#%load_ext dotenv
#%dotenv
import os

token = {
    'access_token': os.getenv("CLIENT_ACCESS_TOKEN"),
    'token_type': 'Bearer'
}

oauth = OAuth2(os.getenv("CLIENT_ID"),
               None,token)

In [7]:
token

{'access_token': 'mBnDnHyEI_KGNyYv39-7efdwjJXc5cWq77fg_eDlsO8RDkYS8uu0YvOiMPS0e-FX',
 'token_type': 'Bearer'}

In [8]:
from pymongo import MongoClient
import requests

client = MongoClient()

In [9]:
client.database_names()

['admin',
 'books',
 'catalog',
 'config',
 'events',
 'gambino',
 'legistlation',
 'local']

In [10]:
db = client.gambino
songs = db.songs

In [11]:
db.songs.count()

3899

In [12]:
# This "user_timeline" url tells the API to get the tweets of the user associated with the app.
response = requests.get("https://api.genius.com/artists/1745/songs?sort=popularity",
                        auth=oauth)

# Note that we're using the "requests" library
cg_songs = response.json()

In [13]:
## "https://api.genius.com/artists/1745"

len(cg_songs)

2

In [14]:
cg_songs

{'meta': {'status': 200},
 'response': {'songs': [{'annotation_count': 20,
    'api_path': '/songs/3688791',
    'full_title': 'This Is America by\xa0Childish\xa0Gambino',
    'header_image_thumbnail_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.300x100x1.jpg',
    'header_image_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.1000x333x1.jpg',
    'id': 3688791,
    'lyrics_owner_id': 104344,
    'lyrics_state': 'complete',
    'path': '/Childish-gambino-this-is-america-lyrics',
    'pyongs_count': 381,
    'song_art_image_thumbnail_url': 'https://images.genius.com/04cc504b70ee9a45062f125275856a4c.300x300x1.jpg',
    'stats': {'hot': False,
     'unreviewed_annotations': 0,
     'concurrents': 11,
     'pageviews': 4248870},
    'title': 'This Is America',
    'title_with_featured': 'This Is America',
    'url': 'https://genius.com/Childish-gambino-this-is-america-lyrics',
    'primary_artist': {'api_path': '/artists/1745',
     'header_image_url': 

In [15]:
first_songs = cg_songs['response']['songs']

first_songs

[{'annotation_count': 20,
  'api_path': '/songs/3688791',
  'full_title': 'This Is America by\xa0Childish\xa0Gambino',
  'header_image_thumbnail_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.300x100x1.jpg',
  'header_image_url': 'https://images.genius.com/25d40b3685e26eed3e943476a29d1763.1000x333x1.jpg',
  'id': 3688791,
  'lyrics_owner_id': 104344,
  'lyrics_state': 'complete',
  'path': '/Childish-gambino-this-is-america-lyrics',
  'pyongs_count': 381,
  'song_art_image_thumbnail_url': 'https://images.genius.com/04cc504b70ee9a45062f125275856a4c.300x300x1.jpg',
  'stats': {'hot': False,
   'unreviewed_annotations': 0,
   'concurrents': 11,
   'pageviews': 4248870},
  'title': 'This Is America',
  'title_with_featured': 'This Is America',
  'url': 'https://genius.com/Childish-gambino-this-is-america-lyrics',
  'primary_artist': {'api_path': '/artists/1745',
   'header_image_url': 'https://images.genius.com/c2c84d21944bce3835b257b1fc400833.1000x210x1.jpg',
   'id': 1

In [16]:
len(first_songs)

20

In [17]:
# This "user_timeline" url tells the API to get the tweets of the user associated with the app.
response = requests.get("https://api.genius.com/artists/1745/songs?sort=popularity&per_page=20&page=2",
                        auth=oauth)

# Note that we're using the "requests" library
cg_songs2 = response.json()

In [18]:
len(cg_songs2)

2

In [19]:
second_songs = cg_songs2['response']['songs']

print(len(second_songs))
second_songs

20


[{'annotation_count': 31,
  'api_path': '/songs/57158',
  'full_title': 'L.E.S. by\xa0Childish\xa0Gambino',
  'header_image_thumbnail_url': 'https://images.genius.com/91362788ed6088d22ac72895642b3e50.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/91362788ed6088d22ac72895642b3e50.1000x1000x1.jpg',
  'id': 57158,
  'lyrics_owner_id': 11524,
  'lyrics_state': 'complete',
  'path': '/Childish-gambino-les-lyrics',
  'pyongs_count': 76,
  'song_art_image_thumbnail_url': 'https://images.genius.com/00196af444633a4108e6dc544358985e.300x300x1.jpg',
  'stats': {'hot': False, 'unreviewed_annotations': 0, 'pageviews': 381355},
  'title': 'L.E.S.',
  'title_with_featured': 'L.E.S.',
  'url': 'https://genius.com/Childish-gambino-les-lyrics',
  'primary_artist': {'api_path': '/artists/1745',
   'header_image_url': 'https://images.genius.com/c2c84d21944bce3835b257b1fc400833.1000x210x1.jpg',
   'id': 1745,
   'image_url': 'https://images.genius.com/5a9c598da4e86fefcb845cbbe74706aa.1000

In [20]:
# This "user_timeline" url tells the API to get the tweets of the user associated with the app.
response = requests.get("https://api.genius.com/artists/1745/songs?sort=popularity&per_page=20&page=3",
                        auth=oauth)

# Note that we're using the "requests" library
cg_songs3 = response.json()

In [21]:
third_songs = cg_songs3['response']['songs']

print(len(third_songs))
third_songs

20


[{'annotation_count': 45,
  'api_path': '/songs/506780',
  'full_title': 'Driving Ms. Daisy by\xa0Logic (Ft.\xa0Childish\xa0Gambino)',
  'header_image_thumbnail_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'header_image_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'id': 506780,
  'lyrics_owner_id': 282633,
  'lyrics_state': 'complete',
  'path': '/Logic-driving-ms-daisy-lyrics',
  'pyongs_count': 136,
  'song_art_image_thumbnail_url': 'https://images.genius.com/2c9956b0499470e6d5e3e5d1e65f0713.300x300x1.jpg',
  'stats': {'hot': False, 'unreviewed_annotations': 1, 'pageviews': 215217},
  'title': 'Driving Ms. Daisy',
  'title_with_featured': 'Driving Ms. Daisy (Ft.\xa0Childish\xa0Gambino)',
  'url': 'https://genius.com/Logic-driving-ms-daisy-lyrics',
  'primary_artist': {'api_path': '/artists/7922',
   'header_image_url': 'https://images.genius.com/f6b08bf607b94a279d743e14bbca2a3a.1000x563x1.jpg',
   'id': 792

In [22]:
third_songs

[{'annotation_count': 45,
  'api_path': '/songs/506780',
  'full_title': 'Driving Ms. Daisy by\xa0Logic (Ft.\xa0Childish\xa0Gambino)',
  'header_image_thumbnail_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'header_image_url': 'https://images.genius.com/47ab70be3f64e7061a5d1f1a590e60ff.220x220x1.jpg',
  'id': 506780,
  'lyrics_owner_id': 282633,
  'lyrics_state': 'complete',
  'path': '/Logic-driving-ms-daisy-lyrics',
  'pyongs_count': 136,
  'song_art_image_thumbnail_url': 'https://images.genius.com/2c9956b0499470e6d5e3e5d1e65f0713.300x300x1.jpg',
  'stats': {'hot': False, 'unreviewed_annotations': 1, 'pageviews': 215217},
  'title': 'Driving Ms. Daisy',
  'title_with_featured': 'Driving Ms. Daisy (Ft.\xa0Childish\xa0Gambino)',
  'url': 'https://genius.com/Logic-driving-ms-daisy-lyrics',
  'primary_artist': {'api_path': '/artists/7922',
   'header_image_url': 'https://images.genius.com/f6b08bf607b94a279d743e14bbca2a3a.1000x563x1.jpg',
   'id': 792

In [23]:
# ONLY RUN THIS WHEN NEW SONGS ARE ADDED
# db.songs.insert_many(first_songs)
# db.songs.insert_many(second_songs)
# db.songs.insert_many(third_songs)

In [24]:
def childish_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/1745/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [25]:
# childish_api_to_mongo(9)

In [26]:
def chance_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/16751/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [27]:
# chance_api_to_mongo(8)

In [28]:
def kdot_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/1421/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [29]:
# kdot_api_to_mongo(16)

In [30]:
def drake_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/130/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [31]:
# drake_api_to_mongo(23)

In [32]:
def kanye_api_to_mongo(pages):
    for i in range(1, pages+1):
        response = requests.get(("https://api.genius.com/artists/72/songs?sort=popularity&per_page=50&page=" + str(i)),
                        auth=oauth)
        songs_json = response.json()
        songs = songs_json['response']['songs']
        db.songs.insert_many(songs)
        print('Page ' + str(i) + ' complete!')

In [33]:
# kanye_api_to_mongo(28)

In [34]:
db.songs.count()

3899

In [35]:
song_list = []

for item in db.songs.find():
    item = (item['primary_artist']['name'])
    print(item)

Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Chance The Rapper
Childish Gambino
Childish Gambino
Childish Gambino
Jhené Aiko
Childish Gambino
Kendrick Lamar
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Chance The Rapper
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Logic
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Iggy Azalea
Childish Gambino
Childish Gambino
Childish Gambino
Regular Show
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childish Gambino
Childi

Thesewergod
René Marban
2oD
Wasted Potency
Azure The Paradox
Brian Fresco
Chance The Rapper
George Mackaveli
Cub Sport
Chance The Rapper
DND
Lucas Mobi & Riko
Chance The Rapper
Fashion Genius
Peter Cottontale
Chance The Rapper
Split .P
Lil Arliss
Nick Bee
Lil wafers
Novocation
Not3s, Sharper, Chance The Rapper
Love
Ethan Rossow
Vanilla Taurus
Kendrick Lamar
A$AP Rocky
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Big Sean
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Travis Scott
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
ScHoolboy Q
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
A$AP Rocky
Jay Rock, Kendrick Lamar, Future & James Blake
Kanye West
Kendrick Lamar & SZA
Kanye West
Kendrick Lamar
Kendrick Lamar
T.I.
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Sia
Kendrick Lamar
The Weeknd
Kendrick Lamar
Kendrick Lamar
Kendrick Lamar
Eminem
Kendrick Lamar
Kendrick Lamar
Ke

Drake
Rick Ross
Drake
YG
Nicki Minaj
Drake
Drake
Drake
Dave
PARTYNEXTDOOR
Drake
Drake
Drake
Drake
Drake
Drake
OB O'Brien
Drake
Drake
Drake
Lil Wayne
Wizkid
Drake
Drake
Drake
Drake
Lil Wayne
Drake
Drake
Drake
Drake
2 Chainz
Drake
Drake
Drake
Drake
Drake
Aaliyah
Lil Wayne
Drake
Drake
Drake
Drake
Lil Wayne
Snoop Lion
Drake
DJ Esco
Drake
Drake
Baka
Drake
Future
21 Savage
Drake
DJ Khaled
Young Money
Drake
Drake
Jhené Aiko
Drake
Majid Jordan
N.E.R.D & Rihanna
2 Chainz
Alicia Keys
Drake
Drake
Drake
Drake
Rick Ross
Nicki Minaj
Drake
Drake
Drake
Kanye West
Rick Ross
Justin Timberlake
Justin Bieber
JoJo
PARTYNEXTDOOR
Drake
Birdman
Mary J. Blige
Future
Drake
Drake
Tyga
Gucci Mane
Drake
Drake
Drake
Tinashe
Wizkid
2 Chainz
Future
The Game
Drake
H.E.R.
Trouble
William Singe & Alex Aiono
Timbaland
Lil Wayne
Timbaland
Future
Drake
Drake
Riz
Waka Flocka Flame
Drake
Lil Wayne
Lil Wayne
PARTYNEXTDOOR
The Weeknd
Drake
Drake
Gucci Mane
Drake
Drake
Jamie Foxx
Tyga
Drake
Drake
Drake
Tinashe
Birdman
DJ Drama


Kanye West
Do or Die
JarJarJr
Dia Frampton
Kanye West
Russell Simmons
Kanye West
Kanye West
Kanye West
Kanye West
Gamemast15r
Kanye West
Consequence
The Cleveland Show
GRAV
Kanye West
GRAV
ESB [Nuevo x Jhutch]
Kanye West
Kanye West
Madonna
Nappy Roots
Talib Kweli
Kanye West
Kanye West
De La Soul
Kanye West
Kanye West
Kanye West
Micah Walton
Nick Cannon
Kanye West
Kanye West
Kanye West
Kanye West
Kanye West
Entertainment Tonight
Jay King (Rapper)
Kanye West
Kanye West
Kanye West
Fashion Genius
Kanye West
Kanye West
Kanye West
Kanye West
Kanye West & Charlie Wilson
FrankJavCee
Kanye West
Tory Lanez
Kanye West
Charlamagne Tha God
Kanye West
Madonna
Kanye West
Kanye West
JP Reynolds
Go Getters
Paco
Cassius Henry
Casey Frey
Kanye West
Cub Sport
Kanye West
Kanye West
Ace Hashimoto
Kanye West
Miranda Cosgrove
Trina & Tamara
Stronghold
Kanye West
Freddie Gibbs
Youngduckgoon
Roc Nation
Kanye West
Jamx
Common
Jet 2
Calez
Kanye West
Freeway and Twista
Kanye West
Do or Die
Kanye West
Kanye West
Ka

In [36]:
artist = [item['primary_artist']['name'] for item in db.songs.find()]
songs = [item['title'] for item in db.songs.find()]
urls = [item['url'] for item in db.songs.find()]

result = zip(artist, songs, urls)
track_list = list(result)
track_list

[('Childish Gambino',
  'This Is America',
  'https://genius.com/Childish-gambino-this-is-america-lyrics'),
 ('Childish Gambino',
  'Redbone',
  'https://genius.com/Childish-gambino-redbone-lyrics'),
 ('Childish Gambino',
  'V. 3005',
  'https://genius.com/Childish-gambino-v-3005-lyrics'),
 ('Childish Gambino',
  'IV. Sweatpants',
  'https://genius.com/Childish-gambino-iv-sweatpants-lyrics'),
 ('Childish Gambino',
  'Bonfire',
  'https://genius.com/Childish-gambino-bonfire-lyrics'),
 ('Childish Gambino',
  'Heartbeat',
  'https://genius.com/Childish-gambino-heartbeat-lyrics'),
 ('Chance The Rapper',
  'Favorite Song',
  'https://genius.com/Chance-the-rapper-favorite-song-lyrics'),
 ('Childish Gambino',
  'Freaks and Geeks',
  'https://genius.com/Childish-gambino-freaks-and-geeks-lyrics'),
 ('Childish Gambino',
  'III. telegraph ave. ("Oakland" by Lloyd)',
  'https://genius.com/Childish-gambino-iii-telegraph-ave-oakland-by-lloyd-lyrics'),
 ('Childish Gambino',
  'I. The Worst Guys',
  '

In [37]:
for track in track_list:
    if track[0] == 'Childish Gambino':
        artist = track[0]
        url = track[2]
        print(list([artist, url]))

['Childish Gambino', 'https://genius.com/Childish-gambino-this-is-america-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-redbone-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-v-3005-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-iv-sweatpants-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-bonfire-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-heartbeat-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-freaks-and-geeks-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-iii-telegraph-ave-oakland-by-lloyd-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-i-the-worst-guys-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-you-see-me-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-me-and-your-mama-lyrics']
['Childish Gambino', 'https://genius.com/Childish-gambino-sober-lyrics']
['Childish Gambino', 'https://

In [38]:
trial_list = [('Childish Gambino', 'https://genius.com/Childish-gambino-redbone-lyrics'),
              ('Childish Gambino', 'https://genius.com/Childish-gambino-v-3005-lyrics')]

In [39]:
def track_info():
    artist = [item['primary_artist']['name'] for item in db.songs.find()]
    songs = [item['title'] for item in db.songs.find()]
    urls = [item['url'] for item in db.songs.find()]
    song_api_url = [('https://api.genius.com' + item['api_path']) for item in db.songs.find()]

    result = zip(artist, songs, urls, song_api_url)
    track_list = list(result)
    return track_list

In [40]:
track_list = track_info()

In [41]:
print(len(set(track_list)))
track_list

3823


[('Childish Gambino',
  'This Is America',
  'https://genius.com/Childish-gambino-this-is-america-lyrics',
  'https://api.genius.com/songs/3688791'),
 ('Childish Gambino',
  'Redbone',
  'https://genius.com/Childish-gambino-redbone-lyrics',
  'https://api.genius.com/songs/2905167'),
 ('Childish Gambino',
  'V. 3005',
  'https://genius.com/Childish-gambino-v-3005-lyrics',
  'https://api.genius.com/songs/242093'),
 ('Childish Gambino',
  'IV. Sweatpants',
  'https://genius.com/Childish-gambino-iv-sweatpants-lyrics',
  'https://api.genius.com/songs/276502'),
 ('Childish Gambino',
  'Bonfire',
  'https://genius.com/Childish-gambino-bonfire-lyrics',
  'https://api.genius.com/songs/52970'),
 ('Childish Gambino',
  'Heartbeat',
  'https://genius.com/Childish-gambino-heartbeat-lyrics',
  'https://api.genius.com/songs/58043'),
 ('Chance The Rapper',
  'Favorite Song',
  'https://genius.com/Chance-the-rapper-favorite-song-lyrics',
  'https://api.genius.com/songs/146855'),
 ('Childish Gambino',
 

In [42]:
for i in track_list:
    print(i[3])

https://api.genius.com/songs/3688791
https://api.genius.com/songs/2905167
https://api.genius.com/songs/242093
https://api.genius.com/songs/276502
https://api.genius.com/songs/52970
https://api.genius.com/songs/58043
https://api.genius.com/songs/146855
https://api.genius.com/songs/4330
https://api.genius.com/songs/247357
https://api.genius.com/songs/269013
https://api.genius.com/songs/219626
https://api.genius.com/songs/56026
https://api.genius.com/songs/3117
https://api.genius.com/songs/2133930
https://api.genius.com/songs/518571
https://api.genius.com/songs/249733
https://api.genius.com/songs/278511
https://api.genius.com/songs/244804
https://api.genius.com/songs/58049
https://api.genius.com/songs/278544
https://api.genius.com/songs/57158
https://api.genius.com/songs/233929
https://api.genius.com/songs/58044
https://api.genius.com/songs/284778
https://api.genius.com/songs/146880
https://api.genius.com/songs/289028
https://api.genius.com/songs/278501
https://api.genius.com/songs/244019

https://api.genius.com/songs/2612
https://api.genius.com/songs/2408424
https://api.genius.com/songs/68760
https://api.genius.com/songs/75333
https://api.genius.com/songs/32142
https://api.genius.com/songs/1891382
https://api.genius.com/songs/2417708
https://api.genius.com/songs/74017
https://api.genius.com/songs/1111
https://api.genius.com/songs/61685
https://api.genius.com/songs/2320647
https://api.genius.com/songs/406
https://api.genius.com/songs/2342141
https://api.genius.com/songs/3558196
https://api.genius.com/songs/395220
https://api.genius.com/songs/67085
https://api.genius.com/songs/76564
https://api.genius.com/songs/155504
https://api.genius.com/songs/1651
https://api.genius.com/songs/5170
https://api.genius.com/songs/1194
https://api.genius.com/songs/2687
https://api.genius.com/songs/2532
https://api.genius.com/songs/2276825
https://api.genius.com/songs/2714
https://api.genius.com/songs/1893
https://api.genius.com/songs/2421969
https://api.genius.com/songs/57543
https://api.g

https://api.genius.com/songs/49694
https://api.genius.com/songs/2428658
https://api.genius.com/songs/1299819
https://api.genius.com/songs/19624
https://api.genius.com/songs/2853290
https://api.genius.com/songs/2833641
https://api.genius.com/songs/17034
https://api.genius.com/songs/25755
https://api.genius.com/songs/2831141
https://api.genius.com/songs/131977
https://api.genius.com/songs/346383
https://api.genius.com/songs/1336015
https://api.genius.com/songs/32381
https://api.genius.com/songs/25781
https://api.genius.com/songs/1224904
https://api.genius.com/songs/1796572
https://api.genius.com/songs/31863
https://api.genius.com/songs/146089
https://api.genius.com/songs/247942
https://api.genius.com/songs/3769109
https://api.genius.com/songs/33458
https://api.genius.com/songs/2853269
https://api.genius.com/songs/141213
https://api.genius.com/songs/450498
https://api.genius.com/songs/14533
https://api.genius.com/songs/139952
https://api.genius.com/songs/40620
https://api.genius.com/songs

In [43]:
## CREATE ALBUMS COLLECTION BEFORE RUNNING!!!!
db = client.gambino
albums = db.albums

db.albums.count()

0

In [44]:
# response = requests.get('https://api.genius.com/songs/247357', auth=oauth)

# songs = []
# albums = []

# a = response.json()
# song_name = a['response']['song']['title']
# album_name = a['response']['song']['album']['name']

# songs.append(song_name)
# albums.append(album_name)

# result = list(zip(songs, albums))

# print(result)

SSLError: HTTPSConnectionPool(host='api.genius.com', port=443): Max retries exceeded with url: /songs/247357 (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')",),))

In [45]:
len(track_list)

3899

In [46]:
def albums_to_mongo(track_list):
    songs = []
    albums = []
    track_number = 1
    skipped = 0
    for i in track_list:
        try:
            response = requests.get((i[3]), auth=oauth)
            a = response.json()
            song_name = a['response']['song']['title']
            album_name = a['response']['song']['album']['name']
            songs.append(song_name)
            albums.append(album_name)
            track_number += 1
            if track_number % 100 == 0:
                print('Song ' + str(track_number) + ' complete!')
        except:
            skipped += 1
            if skipped % 5 == 0:
                print(str(skipped) + ' songs skipped!')

    result = list(zip(songs, albums))
    
    return (result)

In [47]:
albums = albums_to_mongo(track_list)

print(len(albums))

5 songs skipped!
10 songs skipped!
15 songs skipped!
Song 100 complete!
20 songs skipped!
25 songs skipped!
30 songs skipped!
35 songs skipped!
40 songs skipped!
45 songs skipped!
50 songs skipped!
55 songs skipped!
Song 200 complete!
60 songs skipped!
65 songs skipped!
70 songs skipped!
75 songs skipped!
80 songs skipped!
85 songs skipped!
90 songs skipped!
95 songs skipped!
100 songs skipped!
105 songs skipped!
110 songs skipped!
115 songs skipped!
Song 300 complete!
120 songs skipped!
125 songs skipped!
130 songs skipped!
135 songs skipped!
140 songs skipped!
Song 400 complete!
145 songs skipped!
150 songs skipped!
155 songs skipped!
160 songs skipped!
165 songs skipped!
170 songs skipped!
175 songs skipped!
180 songs skipped!
185 songs skipped!
190 songs skipped!
195 songs skipped!
200 songs skipped!
205 songs skipped!
210 songs skipped!
215 songs skipped!
220 songs skipped!
225 songs skipped!
230 songs skipped!
235 songs skipped!
240 songs skipped!
245 songs skipped!
250 songs ski

In [48]:
columns = ['song_title', 'album']

albums_df = pd.DataFrame(albums, columns=columns)

albums_df.head()

,song_title,album
0,This Is America,CG4
1,Redbone,"""Awaken, My Love!"""
2,V. 3005,Because the Internet
3,IV. Sweatpants,Because the Internet
4,Bonfire,Camp


In [50]:
import pickle

with open('data/albums_df.pkl', 'wb') as picklefile:
    pickle.dump(albums_df, picklefile)

In [ ]:
# albums_to_mongo(track_list)

In [13]:
def artist_tracks(artist_name, track_list):
    tracks = []
    for track in track_list:
        if track[0] == artist_name:
            artist = track[0]
            song_name = track[1]
            url = track[2]
            tracks.append([artist, song_name, url])
    return tracks

In [ ]:
childish_tracks = artist_tracks('Childish Gambino', track_list)

In [ ]:
len(childish_tracks)

In [ ]:
childish_tracks

In [14]:
def get_lyrics(track_list):
    all_song_lyrics = []
    artist = []
    for track in track_list:
        artist = track[0]
        url = track[1]
        source_code = requests.get(url)
        soup = BeautifulSoup(source_code.text, 'html5lib')
        lyrics = soup.find('div', class_='lyrics').text.strip()
        lyrics = lyrics.split('\n\n\n')
        lyrics = lyrics.split('\n\n')
        sleep(random.uniform(2.1, 7.5))

        for i in lyrics:
            if i[:2] != '\n':
                verse = i.split('\n')
                if verse == '':
                    song_element = ''
                    lyrics = ''
                else:
                    if verse[0][0] == '[':
                        song_element = verse[0]
                        lyrics = verse[1:]
                    else:
                        song_element = ''
                        lyrics = verse
            else:
                song_element = ''
                lyrics = ''
            all_song_lyrics.append([artist,song_element,lyrics])
            
    return all_song_lyrics

In [21]:
def scrape_lyrics(track_list):
    all_song_lyrics = []
    artist = []
    for track in track_list:
        artist = track[0]
        song_name = track[1]
        url = track[2]
        try:
            source_code = requests.get(url)
            soup = BeautifulSoup(source_code.text, 'html5lib')
            lyrics = soup.find('div', class_='lyrics').text.strip()
        except:
            lyrics =''
        all_song_lyrics.append([artist,song_name,lyrics])
        sleep(random.uniform(0.1, 1.5))
        
    return all_song_lyrics
        
def split_lyrics(lyrics):
    all_song_elements = []
    for i in lyrics:
        artist = i[0]
        song_name = i[1]
        all_lyrics = i[2]
        split_lyrics = all_lyrics.split('\n\n\n')
        split_lyrics = all_lyrics.split('\n\n')

        for i in split_lyrics:
            if i[:2] != '\n':
                verse = i.split('\n')
                if verse == '':
                    song_element = ''
                    only_lyrics = ''
                else:
                    if verse[0][0] == '[':
                        song_element = verse[0]
                        only_lyrics = verse[1:]
                    else:
                        song_element = ''
                        only_lyrics = verse
            else:
                song_element = ''
                only_lyrics = ''
            all_song_elements.append([artist,song_name,song_element,only_lyrics])

    return all_song_elements

In [ ]:
source_code = requests.get('https://genius.com/Childish-gambino-ii-shadows-lyrics')
soup = BeautifulSoup(source_code.text, 'html5lib')
lyrics = soup.find('div', class_='lyrics').text.strip()
lyrics = lyrics.split('\n\n')

lyrics

In [ ]:
all_song_lyrics = []

for i in lyrics:
    if i[:2] != '\n':
        verse = i.split('\n')
        if verse == '':
            song_element = ''
            lyrics = ''
        else:
            if verse[0][0] == '[':
                song_element = verse[0]
                lyrics = verse[1:]
            else:
                song_element = ''
                lyrics = verse
    else:
        song_element = ''
        lyrics = ''
    print(song_element, lyrics)

In [ ]:
lyrics

In [ ]:
# childish_lyrics_first5 = scrape_lyrics(childish_tracks[:5])

In [ ]:
print(len(childish_lyrics_first5))
childish_lyrics_first5

In [ ]:
childish_first5 = split_lyrics(childish_lyrics_first5)

childish_first5

In [ ]:
df = pd.DataFrame(childish_first5)

df

In [ ]:
df[3] = df[3].apply(', '.join)

In [ ]:
df[3]

In [ ]:
df

In [ ]:
# childish_lyrics = scrape_lyrics(childish_tracks)

In [ ]:
print(len(childish_lyrics))
childish_lyrics[:10]

In [ ]:
import pickle

with open('childish_gambino_real.pkl', 'wb') as picklefile:
    pickle.dump(childish_lyrics, picklefile)

In [ ]:
## fix split_lyrics function and split all of the pkl files

In [ ]:
db.songs.count()

In [ ]:
db.songs.count()

In [ ]:
db.songs.count()

In [43]:
db.songs.count()

3899

In [44]:
track_list = track_info()

NameError: name 'track_info' is not defined

In [ ]:
print(len(set(track_list)))

In [ ]:
chance_tracks = artist_tracks('Chance The Rapper', track_list)

print(len(chance_tracks))
chance_tracks[:5]

In [19]:
kdot_tracks = artist_tracks('Kendrick Lamar', track_list)

print(len(kdot_tracks))
kdot_tracks

351


[['Kendrick Lamar',
  'Look Out For Detox',
  'https://genius.com/Kendrick-lamar-look-out-for-detox-lyrics'],
 ['Kendrick Lamar',
  'HUMBLE.',
  'https://genius.com/Kendrick-lamar-humble-lyrics'],
 ['Kendrick Lamar',
  '\u200bm.A.A.d city',
  'https://genius.com/Kendrick-lamar-maad-city-lyrics'],
 ['Kendrick Lamar',
  'Swimming Pools (Drank)',
  'https://genius.com/Kendrick-lamar-swimming-pools-drank-lyrics'],
 ['Kendrick Lamar', 'DNA.', 'https://genius.com/Kendrick-lamar-dna-lyrics'],
 ['Kendrick Lamar',
  'Money Trees',
  'https://genius.com/Kendrick-lamar-money-trees-lyrics'],
 ['Kendrick Lamar',
  "Bitch, Don't Kill My Vibe",
  'https://genius.com/Kendrick-lamar-bitch-dont-kill-my-vibe-lyrics'],
 ['Kendrick Lamar', 'XXX.', 'https://genius.com/Kendrick-lamar-xxx-lyrics'],
 ['Kendrick Lamar',
  'Poetic Justice',
  'https://genius.com/Kendrick-lamar-poetic-justice-lyrics'],
 ['Kendrick Lamar',
  'King Kunta',
  'https://genius.com/Kendrick-lamar-king-kunta-lyrics'],
 ['Kendrick Lamar'

In [ ]:
drake_tracks = artist_tracks('Drake', track_list)

print(len(drake_tracks))
drake_tracks[:5]

In [ ]:
kanye_tracks = artist_tracks('Kanye West', track_list)

print(len(kanye_tracks))
kanye_tracks[:5]

In [ ]:
# chance_lyrics = scrape_lyrics(chance_tracks)

In [ ]:
print(len(chance_lyrics))
chance_lyrics[:10]

In [ ]:
with open('chance_the_rapper.pkl', 'wb') as picklefile:
    pickle.dump(chance_lyrics, picklefile)

In [22]:
# kdot_lyrics = scrape_lyrics(kdot_tracks)

In [23]:
print(len(kdot_lyrics))
kdot_lyrics[:10]

351


[['Kendrick Lamar',
  'Look Out For Detox',
  "[Intro]\nUh, shoutout to ScHoolboy Q\nHe uglier than a motherfucker (laughs)\n\n[Verse]\nTire marks, tire marks\nFinish line with the tire marks\nWhen the relay starts I’m a runaway slave\nUgh, walking on water and running on waves\nGod MC Oh my God you gotta see\nThere’s never no I's in me\nOf an Odyssey, I’m a block away\nFire marshal's moving in\nMarshmallows inside my pen\nSweet sixteens\nGot a sweet sixteen and they deadlier than sin\nI’m so appalled\nWith the prototype with a godly protocol\nYou an amateur, they wanna pro to call\nI damage ya on camera, in Compton, in Canada\nI don’t care where ya are\nJust blink twice and I’m there where you are\nLike a shadow in the dark, you a paddle in the boat\nIn an ocean full of sharks bout to come up short\nWater in the pot, flow crack rock like Bam Bam nigga\nHave two grams nigga pay up or blam blam nigga\nHad a black Camry, bumpin' Dipset, Killa Cam nigga\nI had been around niggas, killas, 

In [25]:
import pickle

with open('kendrick.pkl', 'wb') as picklefile:
    pickle.dump(kdot_lyrics, picklefile)

In [ ]:
# drake_lyrics = scrape_lyrics(drake_tracks)

In [ ]:
print(len(drake_lyrics))
drake_lyrics[:10]

In [ ]:
with open('drake.pkl', 'wb') as picklefile:
    pickle.dump(drake_lyrics, picklefile)

In [ ]:
# kanye_lyrics = scrape_lyrics(kanye_tracks)

In [ ]:
print(len(kanye_lyrics))
kanye_lyrics[:10]

In [ ]:
with open('kanye.pkl', 'wb') as picklefile:
    pickle.dump(kanye_lyrics, picklefile)